In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline
from IPython.display import display

In [ ]:
df_2d = pd.read_csv("performance_vhm_lab_100.csv")
df_2d = df_2d.set_index(df_2d.columns[0])
df_2d= df_2d.sort_index()
display(df_2d)

In [ ]:
def computeSpeedupEfficiency(df):
    """
    Given a datafram with index of the number of threads, and the colums 
    multiple time measurements, computer the derived speed up and efficiency.
    """
    
    for colName in df.columns:
        cloc = df.columns.get_loc(colName)
        
        # time with 1 thread
        t1 = df.iloc[0][colName]
    
        #df["Optimal"] = df_2d.iloc[0,1]/df_2d.index.values*(1.0)
        df.insert(loc=cloc+1,
                     column="Relative Speedup {0}".format(colName),
                     value=t1/df.iloc[:,cloc])
        df.insert(loc=cloc+1,
                     column="Relative Efficiency {0}".format(colName),
                     value=t1/(df.iloc[:,cloc]*df.index))

In [ ]:
computeSpeedupEfficiency(df_2d)
display(df_2d)

In [ ]:
df_3d = pd.read_csv("performance_vhm_lab.csv")
df_3d = df_3d.set_index(df_3d.columns[0])
df_3d= df_3d.sort_index()
display(df_3d)

In [ ]:
computeSpeedupEfficiency(df_3d)
display(df_3d)


In [ ]:
print(df_2d.to_latex(float_format='%.5f'))

In [ ]:
n = 4
a=0.8788
# Efficiency
n**(a-1)

In [ ]:
# Speed up
n**(a)

In [ ]:
# serial part via Gustafson's law
(n**(a)-n)/(1-n)

In [ ]:
# percent parallel via Amdakl's law
(1.0-n**(a))/(n**(a))*(n/(1.0-n))

In [ ]:
print(df_2d.to_latex())
print(df_3d.to_latex())

In [ ]:
df_2d.loc[:44][["Without Connectivity","With Connectivity"]]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, dpi=320, figsize=(10, 10))
#fig.suptitle("SLIC Performance Number of Theads vs Time")

plot_data = df_2d.loc[:44][["Without Connectivity","With Connectivity"]]
ax = plot_data.plot(ax=axes[0,0], marker="")
#ax.set_xscale("log", basex=2)
ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%d'))
#ax.set_yscale("log", basey=2)
ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_ylabel("seconds")
ax.set_title("Time")
ax.legend().set_visible(False)
ax.set_xlabel("")


plot_data = df_2d.loc[44::22][["Without Connectivity","With Connectivity"]]
ax = plot_data.plot.bar(ax=axes[0,1])
ax.legend().set_visible(False)
ax.set_title("HyperThreading")
ax.set_xlabel("")



plot_data = df_3d.loc[:44][["Without Connectivity","With Connectivity"]]
ax = plot_data.plot(ax=axes[1,0], marker="o")
#ax.set_xscale("log", basex=2)
ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%d'))
#ax.set_yscale("log", basey=2)
ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_ylabel("seconds")
#ax.set_title("log-log")
ax.legend(bbox_to_anchor=(.95, .95), loc='upper right', borderaxespad=0.)
#ax.legend().set_visible(False)


plot_data = df_3d.loc[44:][["Without Connectivity","With Connectivity"]]
ax = plot_data.plot.bar(ax=axes[1,1])
ax.legend().set_visible(False)

fig.savefig("thread_time.eps")


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, dpi=320, figsize=(16, 8))

plot_data = df_2d.loc[:44][[col for col in df_2d.columns if "Speedup" in col]]
ax = plot_data.plot(ax=axes[0],)
ax.plot(plot_data.index, plot_data.index,linestyle=':')
ax.legend().set_visible(False)
#ax.set_title("Speedup in 2D")
ax.set_ylabel("speedup")


plot_data = df_3d.loc[:44][[col for col in df_3d.columns if "Speedup" in col]]
ax = plot_data.plot(ax=axes[1], marker="o", label=["A","B"])
#ax.set_title("Speedup in 3D")
ax.plot(plot_data.index,  plot_data.index, label="linear",linestyle=':')
ax.legend(bbox_to_anchor=(.05, .95), loc='upper left', borderaxespad=0., labels=("Without Connectivity","With Connectivity","linear"))
#


fig.savefig("thread_speedup.eps")

In [ ]:
def fit_scalability(x,y):
  x = np.log2(x)
  y = np.sign(y)*np.log2(np.abs(y))
  z = np.polyfit(x=x, y=y, deg=1)
  return -z[0]

In [ ]:
drop=-2
scalability_3d_wo=fit_scalability(df_3d.index[:drop].values,
                                  df_3d.iloc[:drop,0].values)
scalability_3d_w=fit_scalability(df_3d.index[:drop].values,
                                 df_3d.iloc[:drop,1].values)
scalability_3d_c=fit_scalability(df_3d.index[:drop].values,
                                 df_3d.iloc[:drop,1].values-df_3d.iloc[:drop,0].values)
print("3d scalability {0} (without connectivity) {1} (with connectivity) {2} (connectivity alone)"
          .format(scalability_3d_wo,scalability_3d_w,scalability_3d_c))

In [ ]:
drop=-2
scalability_2d_wo=fit_scalability(df_2d.index[:drop].values,
                                  df_2d.iloc[:drop,0].values)
scalability_2d_w=fit_scalability(df_2d.index[:drop].values,
                                 df_2d.iloc[:drop,1].values)
scalability_2d_c=fit_scalability(df_2d.index[:drop].values,
                                 df_2d.iloc[:drop,1].values-df_2d.iloc[:drop,0].values)
print("2d scalability {0} (without connectivity) {1} (with connectivity) {2} (connectivity alone)"
          .format(scalability_2d_wo,scalability_2d_w,scalability_2d_c))

In [ ]:
for i in range(88):
    print(i, end=' ')